In [1]:
import numpy as np
import pandas as pd
import joblib 
import os
import matplotlib.pyplot as plt
import warnings;warnings.simplefilter('ignore')
from numpy.linalg import lstsq,inv

In [2]:
root = '../data/phase_2/cleaned'
os.listdir(root)

['col_names.pkl', 'train.csv']

In [3]:
col_names = joblib.load(root+'/col_names.pkl')

In [4]:
df = pd.read_csv(root+'/train.csv',index_col=0)

In [5]:
df[col_names['case']].head()

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%
001-01,10.0,100.0,2.5,1.0
001-02,10.0,100.0,5.0,1.0
001-03,10.0,100.0,10.0,1.0
001-04,10.0,100.0,2.5,1.5
001-05,10.0,100.0,5.0,1.5


# select df

In [6]:
df = df[(df[col_names['case']].values==[10,100,2.5,1.0]).all(axis=1)].sample(54)
print(df.shape)
df.head()

(54, 386)


,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,...,"Individual Component to Heavy End Split Factor_2,2,3,3-Tetramethylhexane_Ratio",Individual Component to Heavy End Split Factor_3-Methylnonane_Ratio,Individual Component to Heavy End Split Factor_tert-Butylcyclohexane_Ratio,Individual Component to Heavy End Split Factor_n-Decane_Ratio,"Individual Component to Heavy End Split Factor_1,2-Diethylbenzene_Ratio",Individual Component to Heavy End Split Factor_n-Undecane_Ratio,Individual Component to Heavy End Split Factor_n-Pentylbenzene_Ratio,Individual Component to Heavy End Split Factor_n-Dodecane_Ratio,Individual Component to Heavy End Split Factor_n-Hexylbenzene_Ratio,Individual Component to Heavy End Split Factor_n-Tridecane_Ratio
383-01,10.0,100.0,2.5,1.0,0.0,0.0,0.0,0.046000,0.092001,0.766008,...,0.362012,0.386374,0.391481,0.403501,0.417418,0.437303,0.445233,0.453425,0.457469,1.0
154-01,10.0,100.0,2.5,1.0,0.0,0.0,0.0,0.031000,0.061999,0.511995,...,0.340183,0.366367,0.370532,0.383711,0.396474,0.418152,0.425529,0.434644,0.438379,1.0
274-01,10.0,100.0,2.5,1.0,0.0,0.0,0.0,0.060002,0.120005,1.000040,...,0.364936,0.392201,0.397659,0.410745,0.425296,0.447017,0.455196,0.464153,0.468293,1.0
240-01,10.0,100.0,2.5,1.0,0.0,0.0,0.0,0.081004,0.162008,1.348067,...,0.397202,0.427229,0.433080,0.447039,0.462194,0.484885,0.493202,0.502418,0.506582,1.0
353-01,10.0,100.0,2.5,1.0,0.0,0.0,0.0,0.058999,0.117998,0.975980,...,0.388101,0.412079,0.417328,0.429190,0.443284,0.462258,0.470234,0.477890,0.481947,1.0


# get xna fna xle fle

In [7]:
xna = df[col_names['xna']].values
fna = df[col_names['fna']].values
#================================
xle = df[col_names['xle']].values
fle = df[col_names['fle']].values

# AX = B

In [8]:
A = xna
B = xle

In [9]:
X = np.matmul(inv(A),B)
X

LinAlgError: Singular matrix

# lstsq

In [10]:
n_col = [ i.split('_')[-2] for i in col_names['xna']]
def lstsq_sol(A,B):
    x = np.array([np.concatenate((v,[1])) for v in A])
    y = B
    w,error,_,_ = np.linalg.lstsq(x,y)
    y_pred = np.dot(x,w)
    #for i,name in enumerate(n_col):
    #    plt.plot(y[:,i],label='real_{}'.format(name))
    #    plt.plot(y_pred[:,i],label='pred{}'.format(name))
    #    plt.legend()
    #    plt.show()
    res = pd.DataFrame(index=n_col,columns=['MSE'])
    for i,name in enumerate(n_col):
        res.loc[name,'MSE'] = np.sum((y_pred[:,i]-y[:,i])**2)/len(y_pred)
    res.loc['AVG'] = res.mean(axis=0)
    return res

# sle

In [11]:
xle = df[col_names['xle']].values
fle = df[col_names['fle']].values
A = xna
B = xle
res = lstsq_sol(A,B)
res

,MSE
Oxygen,0
Methane,0
Ethane,0
Propane,0.000241528
n-Butane,0.000895779
i-Pentane,0.0561376
n-Pentane,0.224526
tr2-Pentene,3.68961e-05
Cyclopentane,0.0048345
3-Methylpentane,0.0568116


# shc

In [12]:
xhc = df[col_names['xhc']].values
fhc = df[col_names['fhc']].values
A = xna
B = xhc
res = lstsq_sol(A,B)
res

,MSE
Oxygen,0
Methane,0
Ethane,0
Propane,1.58084e-11
n-Butane,3.28828e-10
i-Pentane,1.33559e-07
n-Pentane,1.03993e-06
tr2-Pentene,2.17835e-10
Cyclopentane,1.73578e-07
3-Methylpentane,3.50464e-05


# she

In [13]:
xhe = df[col_names['xhe']].values
fhe = df[col_names['fhe']].values
A = xna
B = xhe
res = lstsq_sol(A,B)
res

,MSE
Oxygen,0
Methane,0
Ethane,0
Propane,0
n-Butane,2.31116e-48
i-Pentane,1.57789e-37
n-Pentane,1.03187e-34
tr2-Pentene,1.22239e-38
Cyclopentane,6.11389e-32
3-Methylpentane,6.19652e-29
